## HW2 Writeup
by Kuan Chieh Peng

## 1. Data Collection
### (a)
ETF holdings: https://www.sectorspdrs.com/ \
Collect the holdings data (underlying and weights) for each of the following Sector Spider ETFs. You may expand into other Spiders if you wish.
- XLE
    - Tracks the Energy Select Sector Index, which is a market-cap-weighted index of the largest publicly traded energy companies in the United States.
- XLF
    - Tracks the Financials Select Sector Index, which is a market-cap-weighted index of the largest publicly traded financial companies in the United States.
- XLC
    - Tracks the Communication Services Select Sector Index, which is a market-cap-weighted index of the largest publicly traded communication services companies in the United States.



In [44]:
import pandas as pd
import yfinance as yf

In [45]:
xle = pd.read_csv("index-holdings-xle.csv")
xlf = pd.read_csv("index-holdings-xlf.csv")
xlc = pd.read_csv("index-holdings-xlc.csv")

xle.columns = xle.iloc[0]
xle = xle.iloc[1:]

xlf.columns = xlf.iloc[0]
xlf = xlf.iloc[1:]
xlc.columns = xlc.iloc[0]
xlc = xlc.iloc[1:]

In [46]:
xlf['Symbol'] = xlf['Symbol'].replace({'BRK.b': 'BRK-b'})
xle.shape, xlf.shape, xlc.shape

((23, 8), (72, 8), (23, 8))

In [47]:
xle.head()

,Symbol,Company Name,Index Weight,Last,Change,%Change,Volume,52 Week Range
1,XOM,Exxon Mobil Corp,23.40%,120.20,+3.79,+3.26%,23.96 M,84.81 - 119.92
2,CVX,Chevron Corp,18.66%,171.04,+3.24,+1.93%,9.12 M,140.46 - 189.68
3,EOG,EOG Resources,4.63%,129.81,+2.93,+2.31%,2.73 M,98.52 - 150.88
4,SLB,Schlumberger Ltd,4.47%,60.80,+1.67,+2.82%,7.66 M,33.86 - 62.78
5,COP,ConocoPhillips,4.45%,123.09,+3.55,+2.97%,5.36 M,91.53 - 138.49


(b) \
Historical price time series, using yfinance package.
1. Install yfinance into your python environment
2. For every constituent of each spider ETF, collect 1 day of intraday price time series at 1-minute interval on the date of your choice

In [48]:
xle_intraday = []
xlf_intraday = []
xlc_intraday = []

xle_symbols = list(xle['Symbol'])
xlf_symbols = list(xlf['Symbol'])
xlc_symbols = list(xlc['Symbol'])

In [49]:
xle_intraday = yf.download(xle_symbols, start = '2023-09-26', end = '2023-09-27', interval = '1m')['Adj Close']
xle_close = yf.download(xle_symbols, start = '2023-09-25', end = '2023-09-26')['Adj Close']

xle_intraday = pd.concat([xle_close, xle_intraday]).reset_index(drop = True)#.fillna(0)

[*********************100%%**********************]  23 of 23 completed
[*********************100%%**********************]  23 of 23 completed


In [50]:
xlf_intraday = yf.download(xlf_symbols, start='2023-09-26', end='2023-09-27', interval='1m')['Adj Close']
xlf_close = yf.download(xlf_symbols, start='2023-09-25', end='2023-09-26')['Adj Close']

xlf_intraday = pd.concat([xlf_close, xlf_intraday]).reset_index(drop = True)#.fillna(0)

[*********************100%%**********************]  72 of 72 completed
[*********************100%%**********************]  72 of 72 completed


In [51]:
xlc_intraday = yf.download(xlc_symbols, start='2023-09-26', end='2023-09-27', interval='1m')['Adj Close']
xlc_close = yf.download(xlc_symbols, start='2023-09-25', end='2023-09-26')['Adj Close']

xlc_intraday = pd.concat([xlc_close, xlc_intraday]).reset_index(drop = True)#.fillna(0)

[*********************100%%**********************]  23 of 23 completed
[*********************100%%**********************]  23 of 23 completed


In [52]:
etf_intraday = yf.download("XLC XLE XLF", start = '2023-09-26', end = '2023-09-27', interval = '1m')['Adj Close']
etf_close = yf.download("XLC XLE XLF", start = '2023-09-25', end = '2023-09-26')['Adj Close']

etf_intraday = pd.concat([etf_close, etf_intraday]).reset_index(drop = True)

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed


## 2. Tracking NAV vs ETF intraday returns
> Generate intraday return spread between ETF and its NAV.


### (a)
First row in xle_intraday is closing price in previous day.

In [53]:
xle_intraday.head()

,APA,BKR,COP,CTRA,CVX,DVN,EOG,EQT,FANG,HAL,...,MRO,OKE,OXY,PSX,PXD,SLB,TRGP,VLO,WMB,XOM
0,41.020000,35.900002,120.465355,26.570000,168.710007,47.509998,125.650002,38.919998,152.639999,40.980000,...,26.610001,66.540001,63.160000,120.260002,225.539993,59.840000,85.320000,146.539993,33.959999,116.230003
1,40.639999,35.639999,119.879997,NaN,167.500000,46.895000,124.769997,38.549999,151.649094,40.665100,...,26.320000,66.300003,62.595001,119.285004,223.664993,59.270000,84.309998,144.065002,33.720001,115.459999
2,40.619999,35.665001,119.900101,26.309999,167.479996,46.962502,124.985001,38.430000,151.929993,40.814999,...,26.330000,66.180000,62.750000,119.260002,223.690002,59.549999,84.385002,143.764999,33.680000,115.470001
3,40.834999,35.764999,120.360001,26.389999,167.732605,47.110001,125.330002,38.645000,152.673904,40.950001,...,26.430000,66.394997,63.009998,119.419998,224.330002,59.660000,84.574997,144.599792,33.709999,115.885002
4,40.794998,35.759998,120.300003,26.440001,167.500000,47.035000,125.514999,38.500000,152.619995,40.959999,...,26.410000,66.489998,62.990002,119.699997,224.279999,59.614201,84.529999,144.589996,33.700001,115.864998


### Difficult approach:

In [54]:
xle_intraday_copy = xle_intraday.copy()

return_list = []

for i in xle_intraday.index[1:]:
    intraday_return = 0

    for col in xle_intraday.columns:
        weight = xle.loc[xle['Symbol'] == col, 'Index Weight'].values[0]
        weight = float(weight.strip('%')) / 100

        share_price = xle_intraday.loc[i, col]
        close_price = xle_intraday.loc[0, col]

        intraday_return += weight * (share_price - close_price) / close_price

    return_list.append(intraday_return)

In [55]:
xle_intraday['Intraday Return'] = ['-'] + return_list
xle_intraday.head()

,APA,BKR,COP,CTRA,CVX,DVN,EOG,EQT,FANG,HAL,...,OKE,OXY,PSX,PXD,SLB,TRGP,VLO,WMB,XOM,Intraday Return
0,41.020000,35.900002,120.465355,26.570000,168.710007,47.509998,125.650002,38.919998,152.639999,40.980000,...,66.540001,63.160000,120.260002,225.539993,59.840000,85.320000,146.539993,33.959999,116.230003,-
1,40.639999,35.639999,119.879997,NaN,167.500000,46.895000,124.769997,38.549999,151.649094,40.665100,...,66.300003,62.595001,119.285004,223.664993,59.270000,84.309998,144.065002,33.720001,115.459999,NaN
2,40.619999,35.665001,119.900101,26.309999,167.479996,46.962502,124.985001,38.430000,151.929993,40.814999,...,66.180000,62.750000,119.260002,223.690002,59.549999,84.385002,143.764999,33.680000,115.470001,-0.007651
3,40.834999,35.764999,120.360001,26.389999,167.732605,47.110001,125.330002,38.645000,152.673904,40.950001,...,66.394997,63.009998,119.419998,224.330002,59.660000,84.574997,144.599792,33.709999,115.885002,-0.004737
4,40.794998,35.759998,120.300003,26.440001,167.500000,47.035000,125.514999,38.500000,152.619995,40.959999,...,66.489998,62.990002,119.699997,224.279999,59.614201,84.529999,144.589996,33.700001,115.864998,-0.004941


In [74]:
for i in xle_intraday.index[]:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Preferred approach:

#### XLE

In [56]:
xle_intraday = xle_intraday_copy.copy()

In [57]:
custom_order = xle_intraday.columns

xle['Symbol'] = pd.Categorical(xle['Symbol'], categories = custom_order, ordered = True)

xle = xle.sort_values('Symbol')
xle = xle.reset_index(drop=True)
xle.head()

,Symbol,Company Name,Index Weight,Last,Change,%Change,Volume,52 Week Range
0,APA,APA Corporation,0.90%,42.38,+1.33,+3.24%,4.19 M,30.67 - 50.58
1,BKR,Baker Hughes Company,2.57%,36.25,+0.54,+1.51%,6.93 M,20.60 - 37.58
2,COP,ConocoPhillips,4.45%,123.09,+3.55,+2.97%,5.36 M,91.53 - 138.49
3,CTRA,Coterra Energy Inc,1.42%,27.14,+0.80,+3.04%,4.94 M,22.25 - 31.76
4,CVX,Chevron Corp,18.66%,171.04,+3.24,+1.93%,9.12 M,140.46 - 189.68


In [58]:
weights = []

for i in xle['Index Weight']:
    weights.append(float(i.strip('%')) / 100)
    
weights[:3]

[0.009000000000000001, 0.025699999999999997, 0.044500000000000005]

In [59]:
xle_intraday_copy = xle_intraday.copy()

xle_intraday_copy = xle_intraday_copy.sub(xle_intraday.iloc[0])
xle_intraday_copy = xle_intraday_copy.div(xle_intraday.iloc[0])
xle_intraday_copy = xle_intraday_copy.mul(weights)
xle_intraday_copy['Intraday Return'] = xle_intraday_copy.sum(axis = 1)

xle_intraday['Intraday Return'] = xle_intraday_copy['Intraday Return']
xle_intraday.head()

,APA,BKR,COP,CTRA,CVX,DVN,EOG,EQT,FANG,HAL,...,OKE,OXY,PSX,PXD,SLB,TRGP,VLO,WMB,XOM,Intraday Return
0,41.020000,35.900002,120.465355,26.570000,168.710007,47.509998,125.650002,38.919998,152.639999,40.980000,...,66.540001,63.160000,120.260002,225.539993,59.840000,85.320000,146.539993,33.959999,116.230003,0.000000
1,40.639999,35.639999,119.879997,NaN,167.500000,46.895000,124.769997,38.549999,151.649094,40.665100,...,66.300003,62.595001,119.285004,223.664993,59.270000,84.309998,144.065002,33.720001,115.459999,-0.007956
2,40.619999,35.665001,119.900101,26.309999,167.479996,46.962502,124.985001,38.430000,151.929993,40.814999,...,66.180000,62.750000,119.260002,223.690002,59.549999,84.385002,143.764999,33.680000,115.470001,-0.007651
3,40.834999,35.764999,120.360001,26.389999,167.732605,47.110001,125.330002,38.645000,152.673904,40.950001,...,66.394997,63.009998,119.419998,224.330002,59.660000,84.574997,144.599792,33.709999,115.885002,-0.004737
4,40.794998,35.759998,120.300003,26.440001,167.500000,47.035000,125.514999,38.500000,152.619995,40.959999,...,66.489998,62.990002,119.699997,224.279999,59.614201,84.529999,144.589996,33.700001,115.864998,-0.004941


#### XLF

In [60]:
custom_order = xlf_intraday.columns

xlf['Symbol'] = pd.Categorical(xlf['Symbol'], categories=custom_order, ordered=True)

xlf = xlf.sort_values('Symbol')
xlf = xlf.reset_index(drop=True)

weights = []

for i in xlf['Index Weight']:
    weights.append(float(i.strip('%')) / 100)

weights[:3]

xlf_intraday_copy = xlf_intraday.copy()

xlf_intraday_copy = xlf_intraday_copy.sub(xlf_intraday.iloc[0])
xlf_intraday_copy = xlf_intraday_copy.div(xlf_intraday.iloc[0])
xlf_intraday_copy = xlf_intraday_copy.mul(weights)
xlf_intraday_copy['Intraday Return'] = xlf_intraday_copy.sum(axis=1)

xlf_intraday['Intraday Return'] = xlf_intraday_copy['Intraday Return']
xlf_intraday.head()

,ACGL,AFL,AIG,AIZ,AJG,ALL,AMP,AON,AXP,BAC,...,TFC,TROW,TRV,USB,V,WFC,WRB,WTW,ZION,Intraday Return
0,81.150002,76.790001,62.130001,147.679993,233.250000,115.239998,335.670013,335.529999,153.229996,27.600000,...,28.840000,105.949997,167.839996,33.590000,233.360001,41.560001,64.910004,213.899994,34.580002,0.000000
1,80.930000,77.059998,61.869999,147.460007,232.429993,114.989998,331.450012,334.369995,151.820007,27.355000,...,28.480000,105.290001,166.639999,33.262798,231.770004,41.049999,64.709999,212.740005,33.915001,-0.007156
2,NaN,77.239998,62.020000,147.354996,NaN,114.739998,NaN,333.940002,151.949997,27.370100,...,28.415899,105.440002,NaN,33.250000,232.119995,41.009998,NaN,NaN,33.869999,-0.008274
3,81.154999,77.345001,62.130001,147.699997,232.654999,115.114998,NaN,NaN,152.100006,27.405001,...,28.469999,105.665001,167.410004,33.299999,231.789993,41.049999,NaN,NaN,34.049999,-0.007080
4,81.370003,77.279999,62.275002,NaN,NaN,115.360001,333.234985,NaN,152.294296,27.450001,...,28.555000,105.690002,NaN,33.395000,232.154999,41.119999,64.830002,NaN,34.180000,-0.005211


#### XLC

In [61]:
custom_order = xlc_intraday.columns  # Assuming you have xlc_intraday

xlc['Symbol'] = pd.Categorical(xlc['Symbol'], categories=custom_order, ordered=True)

xlc = xlc.sort_values('Symbol')
xlc = xlc.reset_index(drop=True)

weights = []

for i in xlc['Index Weight']:
    weights.append(float(i.strip('%')) / 100)

weights[:3]

xlc_intraday_copy = xlc_intraday.copy()

xlc_intraday_copy = xlc_intraday_copy.sub(xlc_intraday.iloc[0])
xlc_intraday_copy = xlc_intraday_copy.div(xlc_intraday.iloc[0])
xlc_intraday_copy = xlc_intraday_copy.mul(weights)
xlc_intraday_copy['Intraday Return'] = xlc_intraday_copy.sum(axis=1)

xlc_intraday['Intraday Return'] = xlc_intraday_copy['Intraday Return']
xlc_intraday.head()

,ATVI,CHTR,CMCSA,DIS,EA,FOX,FOXA,GOOG,GOOGL,IPG,...,NWS,NWSA,OMC,PARA,T,TMUS,TTWO,VZ,WBD,Intraday Return
0,94.050003,440.019989,44.869999,81.010002,119.709999,28.760000,31.209999,132.169998,131.110001,28.879999,...,20.459999,19.790001,74.400002,12.6600,15.06,140.009995,138.750000,33.160000,10.6600,0.000000
1,94.050003,426.899994,43.990002,80.459999,119.209999,28.450001,30.900000,131.022400,129.910004,28.520000,...,20.320000,19.620001,73.809998,12.5000,14.80,139.800003,137.785004,32.884998,10.5500,-0.008971
2,94.050003,430.079987,44.209999,80.470001,119.279999,28.440001,30.889999,130.710007,129.600006,28.549999,...,20.370001,19.690001,73.918999,12.5000,14.79,139.850006,137.779999,32.950100,10.4950,-0.010151
3,94.031998,430.619995,44.220001,80.480003,119.275002,28.420000,30.825001,130.650101,129.565002,28.540001,...,20.340000,19.660000,73.875000,12.4304,14.69,140.059998,137.755005,32.880001,10.4799,-0.010941
4,94.025002,430.100006,44.299999,80.650002,119.250000,28.400000,30.860001,130.440002,129.360001,28.584999,...,20.370001,19.680000,73.849998,12.5000,14.70,140.009995,137.740005,32.834999,10.5250,-0.010998


### (b)
Intraday return of the ETF itself

In [62]:
etf_intraday_copy = etf_intraday.copy()
etf_intraday_copy.head()

,XLC,XLE,XLF
0,65.779999,90.430000,33.740002
1,65.169998,89.650002,33.500000
2,65.099998,89.690002,33.505001
3,65.029999,89.959999,33.529999
4,65.059998,89.940002,33.570000


In [63]:
etf_intraday_copy = etf_intraday_copy.sub(etf_intraday.iloc[0])
etf_intraday_copy = etf_intraday_copy.div(etf_intraday.iloc[0])
etf = etf_intraday_copy.copy()
etf = etf[['XLE', 'XLC', 'XLF']]
etf.head()

,XLE,XLC,XLF
0,0.000000,0.000000,0.000000
1,-0.008625,-0.009273,-0.007113
2,-0.008183,-0.010337,-0.006965
3,-0.005197,-0.011402,-0.006224
4,-0.005419,-0.010946,-0.005039


### (c)
Spread between NAV and ETF:

In [64]:
nav = pd.DataFrame()
nav['XLE'] = xle_intraday['Intraday Return']
nav['XLF'] = xlf_intraday['Intraday Return']
nav['XLC'] = xlc_intraday['Intraday Return']
nav = nav[['XLE', 'XLC', 'XLF']]
nav.head()

,XLE,XLC,XLF
0,0.000000,0.000000,0.000000
1,-0.007956,-0.008971,-0.007156
2,-0.007651,-0.010151,-0.008274
3,-0.004737,-0.010941,-0.007080
4,-0.004941,-0.010998,-0.005211


In [65]:
spread = etf - nav
spread.head()

,XLE,XLC,XLF
0,0.000000,0.000000,0.000000
1,-0.000669,-0.000302,0.000043
2,-0.000532,-0.000186,0.001309
3,-0.000461,-0.000460,0.000856
4,-0.000477,0.000053,0.000172


## 3. Tracking the Spread

### (a)
Set up a DataFrame where each column corresponds to a intraday spread series of an
ETF.

In [66]:
spread.head()

,XLE,XLC,XLF
0,0.000000,0.000000,0.000000
1,-0.000669,-0.000302,0.000043
2,-0.000532,-0.000186,0.001309
3,-0.000461,-0.000460,0.000856
4,-0.000477,0.000053,0.000172


### (b)
Calculate the standard deviation of the spread series

In [67]:
xle_spread = spread['XLE'].std()
xle_spread

9.200080952236231e-05

In [68]:
xlf_spread = spread['XLF'].std()
xlf_spread

0.0010412544387738139

In [69]:
xlc_spread = spread['XLC'].std()
xlc_spread

0.00025434241383266

### (c)
Print out number of incidences for ETF where the spread is outside of 1-std from 0.

In [70]:
xle_count = (etf['XLE'].abs() > xle_spread).sum()
xle_count

389

In [71]:
xlf_count = (etf['XLF'].abs() > xlf_spread).sum()
xlf_count

390

In [72]:
xlc_count = (etf['XLC'].abs() > xlf_spread).sum()
xlc_count

390

The number of incidences for ETF where the spread is outside of 1-std from 0 includes every row. I double checked my calculation and think they should be right. Possibly the NaN values messing with my results.